In [34]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

# ChEA3 Appyter


The ChEA3 Appyter predicts transcription factors (TFs) associated with user-input sets of genes. Discrete query gene sets are compared to ChEA3 libraries of TF target gene sets assembled from multiple orthogonal 'omics' datasets. The Fisher's Exact Test, with a background size of 20,000, is used to compare the input gene set to the TF target gene sets in order to determine which TFs may be most closely associated with the input gene set. 

In [35]:
# imports
import json
import requests
import numpy as np
from time import sleep
from tabulate import tabulate
from IPython.display import HTML, display, Image, FileLink, Markdown
import plotly.graph_objects as go
import kaleido
import os

In [36]:
%%appyter hide_code 

{% do SectionField(
    name = 'input',
    title = 'ChEA Appyter Gene Set Upload',
    subtitle = 'Upload a gene set you wish to analyze',
    img = 'images/chea3.png'
)%}

{% set gene_input = TabField(
    name = 'gene_input',
    label = 'Gene Set Input',
    default = 'Paste',
    description = 'Input your gene set',
    choices = {
        'Paste': [
            TextField(
                name = 'paste_gene_input',
                label = 'Input Gene Set',
                default = 'KIAA0907\nKDM5A\nCDC25A\nEGR1\nGADD45B\nRELB\nTERF2IP\nSMNDC1' +
                '\nTICAM1\nNFKB2\nRGS2\nNCOA3\nTEX10\nARID4B\nCHIC2\nFBXO11\nMTF2\nCDK2' +
                '\nDNTTIP2\nGADD45A\nGOLT1B\nPOLR2K\nNFKBIE\nGABPB1\nECD\nPHKG2\nRAD9A\nNET1' +
                '\nKIAA0753\nEZH2\nNRAS\nATP6V0B\nCDK7\nCCNH\nSENP6\nTIPARP\nFOS\nARPP19' +
                '\nTFAP2A\nKDM5B\nNPC1\nTP53BP2\nNUSAP1\nSCCPDH\nKIF20A\nFZD7\nUSP22\nPIP4K2B' +
                '\nCRYZ\nGNB5\nEIF4EBP1\nPHGDH\nRRAGA\nSLC25A46\nRPA1\nHADH\nDAG1\nRPIA\nP4HA2' +
                '\nMACF1\nTMEM97\nMPZL1\nPSMG1\nPLK1\nSLC37A4\nGLRX\nCBR3\nPRSS23\nNUDCD3' +
                '\nCDC20\nKIAA0528\nNIPSNAP1\nTRAM2\nSTUB1\nDERA\nMTHFD2\nBLVRA\nIARS2\nLIPA' +
                '\nPGM1\nCNDP2\nBNIP3\nCTSL1\nCDC25B\nHSPA8\nEPRS\nPAX8\nSACM1L\nHOXA5\nTLE1' +
                '\nPYGL\nTUBB6\nLOXL1',
                description = 'Input your list of genes (one gene per row)',
                section = 'input'
            )
        ],
        'Upload': [
            FileField(
                name = 'upload_gene_input',
                label = 'Upload File',
                default = '',
                description = 'Upload your list of genes as a text file (one gene per row)',
                section = 'input'
            )
        ]
    },
    section = 'input'
)%}


{% set num_tfs = IntField(
    name = 'num_tfs',
    label = 'Top ranked transcription factors to display',
    description = 'Input the amount of top transcription factors you wish to be returned',
    default = 10,
    min = 1,
    max = 100,
    section = 'input'
)%}


In [37]:
# Function to call the ChEA3 API
def get_chea3_results(gene_set, query_name):
    ADDLIST_URL = 'https://maayanlab.cloud/chea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }
    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok: 
        # r.ok (where r is the object) returns whether the call to the url was successful
        raise Exception('Error analyzing gene list')
    sleep(1)
    return json.loads(response.text) # .text returns the content of response in unicode

# Function for displaying tables 
def display_tables(lib, description):
    for libname in lib:
        display(HTML(f'<h3>{libname}</h3>'))
        
        table = [0] * num_tfs
        tablecounter = 0
        for i in results[libname][0:num_tfs]:
            table[tablecounter] = [i['Rank'],
                                   i['TF'],
                                   f"{i['Intersect']}/{i['Set length']}", 
                                   i['FET p-value'], 
                                   i['FDR'], 
                                   i['Odds Ratio'],
                                   f"{', '.join(i['Overlapping_Genes'].split(',')[0:10])}, ..."]
            tablecounter += 1

        display(HTML(tabulate(table, 
                              ['Rank', 
                               'TF', 
                               'Overlap', 
                               'FET p-value', 
                               'FDR', 
                               'Odds Ratio', 
                               'Overlapping Genes'], 
                              tablefmt='html')))
        
        display(HTML(f'<h5>{description[libname]}</h5>'))
        
        tsv_name = f"{libname.replace(' ', '_')}.tsv"
        with open(tsv_name, 'w') as tsv_file:
            tsv_file.write(tabulate(table, ['Rank', 
                                            'TF',
                                            'Overlap', 
                                            'FET p-value', 
                                            'FDR', 
                                            'Odds Ratio', 
                                            'Overlapping Genes'], 
                                    tablefmt='tsv'))
        display(HTML(f'<a href="{tsv_name}">Download table in .tsv</a>'))
        
        
# Function for displaying the individual library bar charts 
def display_charts(libs, description): 
    for libname in libs:
        
        display(HTML(f'<h3>{libname}</h3>'))
        
        tfs = [i['TF'] for i in results[libname]][0:num_tfs]
        scores = [float(i['FET p-value']) for i in results[libname]][0:num_tfs]
        
        # reverse the order/ranking of the tfs (and their respective scores)
        tfs = tfs[::-1]
        scores = scores[::-1]
        
        # takes the -log of the scores
        scores = -np.log10(scores)
        
        score_range = max(scores) - min(scores)
        x_lowerbound = min(scores) - (score_range * 0.05)
        x_upperbound = max(scores) + (score_range * 0.05)
        
        libfig = go.Figure(data = go.Bar(name = libname, 
                                         x = scores, 
                                         y = tfs, 
                                         marker = go.bar.Marker(color = 'rgb(255,127,80)'), 
                                         orientation = 'h'))
        libfig.update_layout(
            title = {
                'text':'Bar Chart of Scores based on FET p-values',
                'y': 0.87,
                'x': 0.5,
                'xanchor':'center',
                'yanchor':'top'
            },
            xaxis_title = '-log\u2081\u2080(FET p-value)', 
            # \u208 unicode to get the subscript (need a subscript of "10")
            yaxis_title = 'Transcription Factors',
            font = dict(
                size = 16,
                color = 'black'
            )
        )
        
        libfig.update_xaxes(range = [x_lowerbound, x_upperbound])
        
        libfig.show()
        
        display(HTML(f'<h5>{description[libname]}</h5>'))
        
def indexfinder(lib_score_list, value):
    index = 1
    for num in lib_score_list:
        if num == value:
            return index
        elif num != 0:
            index += 1

In [38]:
%%appyter code_exec
# this cell extracts the gene list, whether it was pasted in, or uploaded as a file

{%- if gene_input.raw_value == 'Paste' %} # for if the gene list was pasted in
geneset = {{ gene_input.value[0] }}
{%- else %} # for if the gene list was uploaded as a file (only two options)
geneset_filename = {{ gene_input.value[0] }}
{%- endif %}

# also extracts the integer inputted for number of transcription factors to be returned
num_tfs = {{ num_tfs }}
threshold = 3

```python
# this cell extracts the gene list, whether it was pasted in, or uploaded as a file # for if the gene list was pasted in
geneset = '''KIAA0907
KDM5A
CDC25A
EGR1
GADD45B
RELB
TERF2IP
SMNDC1
TICAM1
NFKB2
RGS2
NCOA3
TEX10
ARID4B
CHIC2
FBXO11
MTF2
CDK2
DNTTIP2
GADD45A
GOLT1B
POLR2K
NFKBIE
GABPB1
ECD
PHKG2
RAD9A
NET1
KIAA0753
EZH2
NRAS
ATP6V0B
CDK7
CCNH
SENP6
TIPARP
FOS
ARPP19
TFAP2A
KDM5B
NPC1
TP53BP2
NUSAP1
SCCPDH
KIF20A
FZD7
USP22
PIP4K2B
CRYZ
GNB5
EIF4EBP1
PHGDH
RRAGA
SLC25A46
RPA1
HADH
DAG1
RPIA
P4HA2
MACF1
TMEM97
MPZL1
PSMG1
PLK1
SLC37A4
GLRX
CBR3
PRSS23
NUDCD3
CDC20
KIAA0528
NIPSNAP1
TRAM2
STUB1
DERA
MTHFD2
BLVRA
IARS2
LIPA
PGM1
CNDP2
BNIP3
CTSL1
CDC25B
HSPA8
EPRS
PAX8
SACM1L
HOXA5
TLE1
PYGL
TUBB6
LOXL1'''
# also extracts the integer inputted for number of transcription factors to be returned
num_tfs = 10
threshold = 3
```

In [39]:
%%appyter code_exec 

# This cell parses the gene list input into an organized python list 
# (when we extracted the gene list, it was still in a text file format, not a python list)

{%- if gene_input.raw_value == 'Paste' %} 
genes = geneset.split('\n')
genes = [x.strip() for x in genes]
{%- else %}
open_gene_input = open(geneset_filename, 'r')
lines = open_gene_input.readlines()
genes = [x.strip() for x in lines]
open_gene_input.close()
{%- endif %}

```python
# This cell parses the gene list input into an organized python list
# (when we extracted the gene list, it was still in a text file format, not a python list)
genes = geneset.split('\n')
genes = [x.strip() for x in genes]
```

In [40]:
# getting results from the ChEA3 API
results = get_chea3_results(genes, 'query')

# so to clarify, "genes" are the user inputted genes (from the input page)
# "results" is the list of transcription factors that is returned by the ChEA3 API in 
# response to the inputted genes

# Libraries

ChEA3 has six different gene set libraries that are composed from multiple sources. ChEA3 also implements two integration techniques to encompass transcription factor analysis across all six of these libraries: Mean Rank and Top Rank. Mean Rank takes the average of a transcription factors individual library scores and ranks according to this mean score. Top Rank takes the best score among a transcription factors individual library scores and ranks according to this top rank.

# Bar Charts

## Mean Rank Bar Chart

In [41]:
c_lib_palette = {'ARCHS4 Coexpression':'rgb(196, 8, 8)',
                 'ENCODE ChIP-seq':'rgb(244, 109, 67)',
                 'Enrichr Queries':'rgb(242, 172, 68)', 
                 'GTEx Coexpression':'rgb(236, 252, 68)',
                 'Literature ChIP-seq':'rgb(165, 242, 162)',
                 'ReMap ChIP-seq':'rgb(92, 217, 78)'}
# this sets all the color values for all the libraries that will be displayed in the bar chart

# NOTE: removed Integrated mean/topRank since those are compiled from the above 6 libraries 
# afterwards and so none of the TFs will have Integrated mean/topRank as one of their libraries

c_lib_means = {'ARCHS4 Coexpression': [0] * num_tfs, 'ENCODE ChIP-seq': [0] * num_tfs, 
               'Enrichr Queries': [0] * num_tfs, 'GTEx Coexpression': [0] * num_tfs,
               'Literature ChIP-seq': [0] * num_tfs, 'ReMap ChIP-seq': [0] * num_tfs}
# creates a dictionary where each library is a key, and the values are empty lists with as
# many indices/spaces as the user has requested transcription factors (ex: if the user
# requests 15 TFs to be returned, the lists will have 15 spaces)


libs_sorted = ['ARCHS4 Coexpression','ENCODE ChIP-seq','Enrichr Queries',
               'GTEx Coexpression','Literature ChIP-seq','ReMap ChIP-seq']



mr_results = results['Integrated--meanRank']
###### NOTE: for meanRank, the TFs are already ranked by Score ######

for i in range(len(mr_results)):
    for lib in libs_sorted:
        mr_results[i].update({lib:0})
        
for i in range(len(mr_results)):
    thing = mr_results[i]['Library'].split(';')
    for a in range(len(thing)):
        library, value = thing[a].split(',')
        mr_results[i].update({library:int(value)})
    
sortedARCHS4 = sorted(mr_results, key = lambda k: k['ARCHS4 Coexpression'])
sortedGTEx = sorted(mr_results, key = lambda k: k['GTEx Coexpression']) 
sortedEnrichr = sorted(mr_results, key = lambda k: k['Enrichr Queries']) 
sortedENCODE = sorted(mr_results, key = lambda k: k['ENCODE ChIP-seq']) 
sortedReMap = sorted(mr_results, key = lambda k: k['ReMap ChIP-seq']) 
sortedLit = sorted(mr_results, key = lambda k: k['Literature ChIP-seq']) 

rankedARCHS4 = [entry['ARCHS4 Coexpression'] for entry in sortedARCHS4]
rankedENCODE = [entry['ENCODE ChIP-seq'] for entry in sortedENCODE]
rankedEnrichr = [entry['Enrichr Queries'] for entry in sortedEnrichr] 
rankedGTEx = [entry['GTEx Coexpression'] for entry in sortedGTEx]
rankedLit = [entry['Literature ChIP-seq'] for entry in sortedLit]
rankedReMap = [entry['ReMap ChIP-seq'] for entry in sortedReMap] 


ranking_dict = {'ARCHS4 Coexpression':rankedARCHS4,
                'ENCODE ChIP-seq':rankedENCODE,
                'Enrichr Queries':rankedEnrichr,
                'GTEx Coexpression':rankedGTEx,
                'Literature ChIP-seq':rankedLit,
                'ReMap ChIP-seq':rankedReMap}

for tfentry in mr_results:
    tfentry.update( [('SumRank', 0), ('AvgRank', 0) ])
    library_scores = tfentry['Library'].split(';')
    lib_counter = 0
    for a in library_scores:
        l, v = a.split(',')
        v = int(v)
        #scorerank = ranking_dict[l].index(v) + 1
        scorerank = indexfinder(ranking_dict[l], int(v))
        tfentry['SumRank'] += int(scorerank)
        lib_counter += 1
    tfentry['AvgRank'] = (tfentry['SumRank'] / lib_counter)
    
sorted_results = sorted(mr_results, key = lambda k: k['AvgRank'])

In [42]:
sorted_top_results = []
index = 0
while (len(sorted_top_results) < num_tfs):
    if len(mr_results[index]['Library'].split(';')) >= threshold:
        sorted_top_results.append(mr_results[index])
    index += 1
    # moves on to the next index
    
sorted_top_results = sorted_top_results[::-1]

# set up a list with all the TFs, sorted by rank (lowest to highest, in line with top_results)
sorted_tfs = []
for i in range(0, len(sorted_top_results)):
    sorted_tfs.append(sorted_top_results[i].get('TF'))
    # this pulls only the TF name from top_results and adds it to sorted_tfs

In [43]:
for i, tfentry in enumerate(sorted_top_results):
    libscores = tfentry['Library'].split(';')
    for a in libscores:
        lib, value = a.split(',')
        #rank = ranking_dict[lib].index(int(value)) + 1
        rank = indexfinder(ranking_dict[lib], int(value))
        avg = tfentry['AvgRank']
        tot = tfentry['SumRank']
        bar_length = (rank*avg)/tot
        c_lib_means[lib][i] = float(bar_length)

In [44]:
# Plotting the actual bar chart
fig = go.Figure(data = [go.Bar(name = c_lib, 
                               x = c_lib_means[c_lib], 
                               y = sorted_tfs,
                               marker = go.bar.Marker(color = c_lib_palette[c_lib]), 
                               orientation = 'h') 
                        for c_lib in libs_sorted])

fig.update_layout(barmode = 'stack')
fig.update_layout(
    title = {
        'text': 'Stacked Bar Chart of Sum of Ranks in Different Libraries',
        'y': 0.87,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title = 'Sum of Ranks in Different Libraries',
    yaxis_title = 'Transcription Factors',
    font = dict(
        size = 16,
        color = 'black'
    )
)

fig.show()

# Graph Info/Organization
# the top ranked TF is listed as the first TF (from top to bottom) on the bar chart 

##### Figure 1. Horizontal bar chart, y-axis represents transcription factors. Displays the top ranked transcription factors according to their average integrated scores across all the libraries.

## Top Rank Bar Chart

In [45]:
tr_sortedtopresults = sorted(results['Integrated--topRank'], key = lambda k: k['Score']) 
tr_sortedtopresults = tr_sortedtopresults[0:num_tfs]
tr_sortedtopresults = tr_sortedtopresults[::-1]

topranklibscores = []
for i in range(num_tfs):
    topranklibscores.append(tr_sortedtopresults[i]['Score'])

tr_lib_scores = {'ARCHS4 Coexpression': [0] * num_tfs, 'ENCODE ChIP-seq': [0] * num_tfs, 
                 'Enrichr Queries': [0] * num_tfs, 'GTEx Coexpression': [0] * num_tfs,
                 'Literature ChIP-seq': [0] * num_tfs, 'ReMap ChIP-seq': [0] * num_tfs}

for i, tfentry in enumerate (tr_sortedtopresults):
    libscores = tfentry['Library'].split(';')
    for value in libscores:
        lib, value = value.split(',')
        tr_lib_scores[lib][i] = float(value)

tr_sortedtfs = []
for i in range(0, len(tr_sortedtopresults)):
    tr_sortedtfs.append(tr_sortedtopresults[i].get('TF'))


tr_fig = go.Figure(data = [go.Bar(name = c_lib, 
                                  x = tr_lib_scores[c_lib], 
                                  y = tr_sortedtfs,
                                  marker = go.bar.Marker(color = c_lib_palette[c_lib]), 
                                  orientation = 'h') 
                           for c_lib in libs_sorted])
tr_fig.update_layout(barmode = 'stack')
tr_fig.update_layout(
    title = {
        'text': 'Stacked Bar Chart of Sum of Ranks in Different Libraries',
        'y': 0.87,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title = 'Sum of Ranks in Different Libraries',
    yaxis_title = 'Transcription Factors',
    font = dict(
        size = 16,
        color = 'black'
    )
)

tr_fig.show()

##### Figure 2. Horizontal bar chart, y-axis represents transcription factors. Displays the top ranked transcription factors according to their top integrated score across all the libraries.

# Individual Library Bar Charts

In [46]:
chart_des = {'GTEx--Coexpression':'Figure 3. Horizontal bar chart, y-axis represents ' +
             'transcription factors. Data mined from the Trascription factors ranked by ' +
             'the GTEx Coexpression gene set library. Data from the TF-target coexpression ' +
             'data in the GTEx dataset.',
             'ReMap--ChIP-seq':'Figure 4. Hortizontal bar chart, y-axis represents ' + 
             'transcription factors. Transcription factors ranked by the ReMap ChIP seq ' +  
             'gene set library. Interaction data mined from the ReMap Project',
             'Enrichr--Queries':'Figure 5. Horizontal bar chart, y-axis represents ' +
             'transcription factors. Transcription factors ranked by the Enrichr Query ' +
             'gene set library. Data from the TF-target co-occurrence data in Enrichr queries.', 
             'ENCODE--ChIP-seq':'Figure 6. Horizontal bar chart, y-axis represents ' +
             'transcription factors. Transcription factors ranked by the ENCODE ChIP seq ' +
             'gene set library. Interaction data mined from the ENCODE project', 
             'ARCHS4--Coexpression':'Figure 7. Horizontal bar chart, y-axis represents ' +
             'transcription factors. Transcription factors evaluated by the ARCHS4 ' +
             'Coexpression gene set library. Data from the TF-target coexpression data ' +
             'in the ARCHS4 dataset.', 
             'Literature--ChIP-seq':'Figure 8. Horizontal bar chart, y-axis represents ' +
             'transcription factors. Transcription factor significance evaluated by the ' +
             'Literature ChIP seq gene set library. Interaction data mined from literature.'}

display_charts(['GTEx--Coexpression',
                'ReMap--ChIP-seq', 
                'Enrichr--Queries',
                'ENCODE--ChIP-seq',
                'ARCHS4--Coexpression',
                'Literature--ChIP-seq'],
              chart_des)

# Tables

## Mean Rank

In [47]:
# first set up an empty list to be filled (eventually we'll be making a table out of this list)
mrtable = [0] * num_tfs

mrcounter = 0
# set up a counter for indexing (to fill mrtable)

for i in results['Integrated--meanRank'][0:num_tfs]:
    mrtable[mrcounter] = [i['Rank'], 
                          i['TF'], 
                          i['Score'], 
                          i['Library'].replace(',', ': ').replace(';', ', '),
                          f"{', '.join(i['Overlapping_Genes'].split(',')[0:10])}, ..."]
    # filling mrtable, using replace() to reformat a bit
    # [0:10] for Overlapping_Genes so that only 10 Overlapping Genes are shown (with '...' after)
    mrcounter += 1

# now actually making/printing the table
display(HTML(tabulate(mrtable, 
                      ['Rank', 
                       'Transcription Factor', 
                       'Mean Rank', 
                       'Libraries Ranks',
                       'Overlapping Genes'], 
                      tablefmt='html')))

Rank,Transcription Factor,Mean Rank,Libraries Ranks,Overlapping Genes
1,RLF,22.33,"ARCHS4 Coexpression: 15, Enrichr Queries: 22, GTEx Coexpression: 30","KDM5A, KDM5B, MACF1, EGR1, CHIC2, GADD45B, GADD45A, TIPARP, NCOA3, BNIP3, ..."
2,CENPA,26.5,"ARCHS4 Coexpression: 31, GTEx Coexpression: 22","CDC20, NRAS, TEX10, PLK1, NUSAP1, KIF20A, CDC25A, EZH2, ..."
3,FOXM1,29.5,"Literature ChIP-seq: 6, ARCHS4 Coexpression: 63, ENCODE ChIP-seq: 51, Enrichr Queries: 33, ReMap ChIP-seq: 7, GTEx Coexpression: 17","KDM5A, ARID4B, TMEM97, CDC20, TUBB6, NRAS, NUSAP1, DAG1, PHGDH, PIP4K2B, ..."
4,E2F7,29.8,"Literature ChIP-seq: 53, ARCHS4 Coexpression: 18, Enrichr Queries: 55, ReMap ChIP-seq: 11, GTEx Coexpression: 12","EGR1, HSPA8, GOLT1B, GADD45A, USP22, NCOA3, PLK1, FOS, TICAM1, TRAM2, ..."
5,E2F1,39.83,"Literature ChIP-seq: 7, ARCHS4 Coexpression: 80, ENCODE ChIP-seq: 42, Enrichr Queries: 51, ReMap ChIP-seq: 46, GTEx Coexpression: 13","KDM5B, CCNH, TMEM97, PRSS23, CDC20, SENP6, TUBB6, NUDCD3, NUSAP1, PHGDH, ..."
6,AHRR,54.5,"ARCHS4 Coexpression: 82, GTEx Coexpression: 27","CDC20, NRAS, PLK1, NUSAP1, TICAM1, NFKBIE, TRAM2, CDC25A, LOXL1, EZH2, ..."
7,ZNF367,61.67,"ARCHS4 Coexpression: 99, Enrichr Queries: 72, GTEx Coexpression: 14","EGR1, NCOA3, PLK1, RPA1, TMEM97, FOS, CDC25A, CDC20, NRAS, MTHFD2, ..."
8,BATF3,63.67,"ARCHS4 Coexpression: 26, Enrichr Queries: 141, GTEx Coexpression: 24","EGR1, GADD45B, GADD45A, NCOA3, FOS, GABPB1, TICAM1, RELB, NFKB2, RGS2, ..."
9,MYBL2,67,"Literature ChIP-seq: 15, ARCHS4 Coexpression: 162, ENCODE ChIP-seq: 5, Enrichr Queries: 32, ReMap ChIP-seq: 165, GTEx Coexpression: 23","KDM5A, KDM5B, GOLT1B, GLRX, TMEM97, EPRS, TRAM2, ARPP19, CDC20, NUDCD3, ..."
10,GTF2B,70,"ARCHS4 Coexpression: 112, ENCODE ChIP-seq: 44, ReMap ChIP-seq: 19, GTEx Coexpression: 105","KDM5B, CHIC2, PYGL, TMEM97, GABPB1, ARPP19, CDC20, SENP6, TUBB6, NUSAP1, ..."


##### Table 1. Displays the top specified number of transcription factors according to their average integrated rank across all the other libraries. Rows represent transcription factors, columns display data associated with the transcription factors.

## Top Rank

In [48]:
trtable = [0] * num_tfs
trcounter = 0
# set up a counter for indexing (to fill mrtable)

for i in results['Integrated--topRank'][0:num_tfs]:
    trtable[trcounter] = [i['Rank'], 
                          i['TF'], 
                          i['Score'], 
                          i['Library'].replace(',', ': ').replace(';', ', '),
                          f"{', '.join(i['Overlapping_Genes'].split(',')[0:10])}, ..."]
    # filling mrtable, using replace() to reformat a bit
    # [0:10] for Overlapping_Genes so that only 10 Overlapping Genes are shown (with '...' after)
    trcounter += 1

    
    
# now actually making/printing the table
display(HTML(tabulate(trtable, 
                      ['Rank', 
                       'Transcription Factor', 
                       'TopRank', 
                       'Libraries Ranks',
                       'Overlapping Genes'], 
                      tablefmt='html')))

Rank,Transcription Factor,TopRank,Libraries Ranks,Overlapping Genes
1,ATF3,0.0006143,ARCHS4 Coexpression: 6.143E-4,"EGR1, NPC1, GADD45B, GADD45A, TIPARP, FOS, TICAM1, NFKBIE, LIPA, RELB, ..."
2,ZNF460,0.0006223,GTEx Coexpression: 6.223E-4,"RPIA, TIPARP, NCOA3, ECD, CDK2, DNTTIP2, GABPB1, NFKBIE, SMNDC1, NFKB2, ..."
3,ZNF586,0.0007123,Enrichr Queries: 7.123E-4,"KDM5B, MACF1, GOLT1B, GLRX, TMEM97, PYGL, EPRS, ARPP19, LOXL1, CNDP2, ..."
4,KLF10,0.001229,ARCHS4 Coexpression: 0.001229,"EGR1, GADD45B, GADD45A, TIPARP, DNTTIP2, FOS, TICAM1, NIPSNAP1, ..."
5,ZBTB21,0.001245,GTEx Coexpression: 0.001245,"KDM5A, GADD45B, CCNH, TIPARP, ECD, CDK2, GABPB1, SMNDC1, HOXA5, ..."
6,HMG20B,0.001425,Enrichr Queries: 0.001425,"KDM5B, EGR1, HSPA8, GADD45B, GADD45A, TIPARP, NCOA3, PLK1, GLRX, PYGL, ..."
7,TGIF2,0.001843,ARCHS4 Coexpression: 0.001843,"FZD7, USP22, MTF2, PLK1, NUSAP1, PHGDH, TMEM97, CDC25A, ..."
8,HIVEP1,0.001867,GTEx Coexpression: 0.001867,"KDM5A, NCOA3, CDK2, GABPB1, TICAM1, NFKBIE, SMNDC1, RELB, NFKB2, ..."
9,ZNF274,0.002137,Enrichr Queries: 0.002137,"KDM5B, MACF1, EGR1, HSPA8, TLE1, GADD45B, GADD45A, TIPARP, NCOA3, FZD7, ..."
10,SNAI2,0.002457,ARCHS4 Coexpression: 0.002457,"TFAP2A, TUBB6, P4HA2, PRSS23, TRAM2, LOXL1, MPZL1, ..."


##### Table 2. Displays the top specified number of transcription factors according to the top integrated rank across all the libraries. Rows represent transcription factors, columns display data associated with the transcription factors.

In [49]:
display(HTML('<h2>The following tables display the rankings of the top' +
             ' transcription factors from each individual library.</h2>'))

table_des = {'GTEx--Coexpression':'Table 3. Displays the top specified number of ' +
             'transcription factors according to TF-target coexpression data from the ' +
             'GTEx dataset. Rows represent transcription factors, columns display data ' +
             'associated with the transcription factors.',
             'ReMap--ChIP-seq':'Table 4. Displays the top specified number of transcription ' +
             'factors according to interaction data mined from the ReMap project. Rows ' +
             'represent transcription factors, columns display data associated with the ' +
             'transcription factors.',
             'Enrichr--Queries':'Table 5. Displays the top specified number of transcription ' +
             'factors according to TF-target co-occurrence data from Enrichr queries. Rows ' +
             'represent transcription factors, columns display data associated with the ' +
             'trasncription factors.', 
             'ENCODE--ChIP-seq':'Table 6. Displyas the top specified number of transcription ' +
             'factors according to interaction data mined from the ENCODE project. Rows ' +
             'represent the transcription factors, columns display data associated with the ' +
             'transcription factors.', 
             'ARCHS4--Coexpression':'Table 7. Displays the top specified number of ' +
             'transcription factors according to TF-target coexpression data from  the ' +
             'ARCHS4 dataset. Rows represent the transcription factors, columns display data ' +
             'associated with the transcription factors', 
             'Literature--ChIP-seq':'Table 8. Displays the top specified number of ' +
             'transcription factors according to interaction data mined from literature. ' +
             'The rows represent the transcription factors, columns display data associated ' +
             'with the transcription factors.'}

display_tables(['GTEx--Coexpression', 
                'ReMap--ChIP-seq', 
                'Enrichr--Queries',
                'ENCODE--ChIP-seq',
                'ARCHS4--Coexpression',
                'Literature--ChIP-seq'], 
               table_des)

Rank,TF,Overlap,FET p-value,FDR,Odds Ratio,Overlapping Genes
1,ZNF460,10/299,2.225e-06,0.00358,7.655,"GABPB1, NCOA3, DNTTIP2, SMNDC1, TIPARP, ECD, NFKB2, RPIA, CDK2, NFKBIE, ..."
2,ZBTB21,9/299,1.571e-05,0.00518,6.866,"KDM5A, CCNH, GABPB1, HOXA5, SMNDC1, GADD45B, TIPARP, ECD, CDK2, ..."
3,HIVEP1,9/300,1.612e-05,0.00518,6.842,"KDM5A, GABPB1, NCOA3, TICAM1, RELB, SMNDC1, NFKB2, CDK2, NFKBIE, ..."
4,NFIL3,9/300,1.612e-05,0.00518,6.842,"CHIC2, CCNH, GABPB1, DNTTIP2, RELB, SMNDC1, GADD45B, GADD45A, TIPARP, ..."
5,NFKB1,9/300,1.612e-05,0.00518,6.842,"GABPB1, NUSAP1, NCOA3, RELB, NRAS, SMNDC1, NFKB2, CDK2, NFKBIE, ..."
6,ZUP1,8/300,0.0001031,0.0276,6.061,"GABPB1, NUSAP1, NCOA3, NRAS, SMNDC1, CDK2, NFKBIE, EZH2, ..."
7,BACH1,7/298,0.0005701,0.0529,5.321,"KDM5A, CHIC2, GABPB1, DNTTIP2, RELB, SMNDC1, CDK7, ..."
8,MAFF,7/298,0.0005701,0.0529,5.321,"CCNH, TICAM1, SMNDC1, GADD45B, GADD45A, TIPARP, CDK2, ..."
9,BAZ2A,7/299,0.0005814,0.0529,5.303,"KDM5A, KDM5B, ARID4B, RRAGA, MTF2, POLR2K, STUB1, ..."
10,ZNF675,7/299,0.0005814,0.0529,5.303,"NUSAP1, MTF2, NRAS, TFAP2A, PLK1, CDK7, EZH2, ..."


Rank,TF,Overlap,FET p-value,FDR,Odds Ratio,Overlapping Genes
1,KMT2A,18/1448,0.0003155,0.0937,2.785,"ARID4B, GABPB1, LIPA, CNDP2, SENP6, NUSAP1, PGM1, HOXA5, FBXO11, RRAGA, ..."
2,GLYR1,15/1442,0.004411,0.447,2.325,"CCNH, EPRS, NUSAP1, CDC25A, RRAGA, MTHFD2, TERF2IP, KIF20A, NRAS, PHKG2, ..."
3,TCF7,15/1446,0.004517,0.447,2.319,"ATP6V0B, NCOA3, MTF2, BLVRA, ARPP19, TP53BP2, EGR1, HSPA8, GADD45A, TIPARP, ..."
4,TEAD1,13/1448,0.02121,0.917,2.004,"CHIC2, PYGL, TRAM2, RGS2, TUBB6, DAG1, FOS, NPC1, GLRX, TFAP2A, ..."
5,TEAD4,13/1449,0.02131,0.917,2.003,"KDM5B, TRAM2, DAG1, NCOA3, FOS, CDC25B, MPZL1, TFAP2A, GADD45B, GADD45A, ..."
6,GATA3,13/1450,0.02141,0.917,2.001,"KDM5B, PYGL, PRSS23, TRAM2, LOXL1, TLE1, NCOA3, FOS, GLRX, POLR2K, ..."
7,FOXM1,13/1452,0.02161,0.917,1.999,"CDC20, NUSAP1, NCOA3, FOS, CDC25B, MTHFD2, KIF20A, TFAP2A, GADD45B, PLK1, ..."
8,CEBPG,12/1438,0.04032,0.994,1.862,"PRSS23, HADH, FOS, MTHFD2, GLRX, NRAS, PHKG2, TFAP2A, HSPA8, GADD45A, ..."
9,STAT3,12/1454,0.04307,0.994,1.841,"HOXA5, FOS, TICAM1, CDC25B, GLRX, TFAP2A, GADD45B, GADD45A, NFKB2, STUB1, ..."
10,ZBTB16,11/1440,0.07621,0.994,1.703,"KDM5A, CHIC2, CDC20, PHGDH, DNTTIP2, CDC25B, RRAGA, MTHFD2, SMNDC1, EGR1, ..."


Rank,TF,Overlap,FET p-value,FDR,Odds Ratio,Overlapping Genes
1,ZNF586,31/298,6.237e-30,8.76e-27,25.68,"KDM5B, GOLT1B, TMEM97, PYGL, EPRS, LOXL1, CNDP2, CDC20, RGS2, NUSAP1, ..."
2,HMG20B,23/298,5.18e-20,3.64e-17,18.5,"KDM5B, PYGL, EPRS, CDC20, RGS2, NUSAP1, PHGDH, HADH, NCOA3, FOS, ..."
3,ZNF274,22/298,7.732e-19,3.62e-16,17.63,"KDM5B, PYGL, EPRS, PRSS23, RGS2, TUBB6, TLE1, NCOA3, FOS, NPC1, ..."
4,TCFL5,20/296,1.345e-16,4.58e-14,16.03,"KDM5B, PYGL, EPRS, PRSS23, CDC20, RGS2, NUSAP1, PHGDH, NCOA3, FOS, ..."
5,ATF1,20/299,1.631e-16,4.58e-14,15.86,"KDM5B, EPRS, CDC20, RGS2, NCOA3, DNTTIP2, FOS, CDC25A, MTHFD2, MTF2, ..."
6,RFX5,19/299,2.146e-15,5.02e-13,15.01,"KDM5B, EPRS, CDC20, RGS2, NUSAP1, PHGDH, NCOA3, FOS, CDC25B, MTHFD2, ..."
7,TP53,18/295,2.145e-14,3.07e-12,14.38,"EPRS, CDC20, RGS2, PHGDH, NCOA3, FOS, CDC25A, MTHFD2, NRAS, TFAP2A, ..."
8,ATF5,18/297,2.407e-14,3.07e-12,14.27,"EPRS, PRSS23, CDC20, RGS2, TUBB6, PHGDH, NCOA3, FOS, MTHFD2, GLRX, ..."
9,FOSL1,18/297,2.407e-14,3.07e-12,14.27,"PRSS23, CDC20, RGS2, TUBB6, NUSAP1, NCOA3, FOS, CDC25A, MTHFD2, KIF20A, ..."
10,ATMIN,18/297,2.407e-14,3.07e-12,14.27,"KDM5B, EPRS, CNDP2, RGS2, TUBB6, PIP4K2B, TLE1, NCOA3, SACM1L, FOS, ..."


Rank,TF,Overlap,FET p-value,FDR,Odds Ratio,Overlapping Genes
1,MAX,28/1825,3.358e-07,0.000185,3.447,"GOLT1B, TMEM97, GABPB1, EPRS, CNDP2, CDC20, NUDCD3, HADH, SLC37A4, HOXA5, ..."
2,RELA,14/769,2.936e-05,0.00546,4.102,"ARID4B, EPRS, NUSAP1, TICAM1, NPC1, MTF2, RELB, CRYZ, NRAS, HSPA8, ..."
3,USF1,26/2113,2.968e-05,0.00546,2.756,"CHIC2, TMEM97, GABPB1, CNDP2, SENP6, NUDCD3, HADH, SLC37A4, HOXA5, SLC25A46, ..."
4,MBD4,22/1672,3.97e-05,0.00548,2.95,"KDM5A, GABPB1, EPRS, PIP4K2B, SACM1L, DNTTIP2, FOS, RRAGA, MTF2, TERF2IP, ..."
5,MYBL2,20/1487,6.04e-05,0.00586,3.016,"KDM5A, KDM5B, EPRS, TRAM2, CDC20, NUDCD3, NUSAP1, SLC37A4, TEX10, DNTTIP2, ..."
6,SP1,21/1611,6.374e-05,0.00586,2.922,"KDM5B, GOLT1B, CCNH, TMEM97, EPRS, LOXL1, CDC20, NUSAP1, PIP4K2B, SLC37A4, ..."
7,E2F4,16/1071,8.776e-05,0.00692,3.355,"GOLT1B, TMEM97, CDC20, TUBB6, NUSAP1, PHGDH, HADH, CDC25A, MTF2, KIF20A, ..."
8,TCF7L2,22/1832,0.0001373,0.00848,2.689,"KDM5A, CHIC2, PRSS23, TRAM2, NCOA3, SACM1L, DNTTIP2, CDC25A, RRAGA, NPC1, ..."
9,PBX3,23/1958,0.0001383,0.00848,2.63,"KDM5A, KDM5B, GOLT1B, GABPB1, PIP4K2B, ATP6V0B, FOS, CDC25A, MTHFD2, MTF2, ..."
10,NFYB,22/1852,0.0001585,0.00875,2.66,"ARID4B, LIPA, CNDP2, CDC20, RGS2, NUSAP1, PHGDH, HADH, HOXA5, SACM1L, ..."


Rank,TF,Overlap,FET p-value,FDR,Odds Ratio,Overlapping Genes
1,ATF3,11/299,2.89e-07,0.000471,8.449,"LIPA, FOS, TICAM1, NPC1, RELB, EGR1, GADD45B, GADD45A, TIPARP, NFKB2, ..."
2,KLF10,8/299,0.0001008,0.056,6.082,"DNTTIP2, FOS, TICAM1, NIPSNAP1, EGR1, GADD45B, GADD45A, TIPARP, ..."
3,TGIF2,8/300,0.0001031,0.056,6.061,"TMEM97, NUSAP1, PHGDH, CDC25A, MTF2, FZD7, USP22, PLK1, ..."
4,SNAI2,7/297,0.000559,0.0689,5.34,"PRSS23, TRAM2, LOXL1, TUBB6, MPZL1, TFAP2A, P4HA2, ..."
5,JUN,7/298,0.0005701,0.0689,5.321,"EPRS, FOS, MTHFD2, EGR1, GADD45B, GADD45A, TIPARP, ..."
6,PRDM9,7/299,0.0005814,0.0689,5.303,"PYGL, EPRS, PHGDH, HOXA5, CDC25A, MTHFD2, IARS2, ..."
7,ZBTB9,7/299,0.0005814,0.0689,5.303,"CDC20, CDC25A, NIPSNAP1, NRAS, HSPA8, FZD7, PLK1, ..."
8,ETV4,7/300,0.0005928,0.0689,5.285,"TMEM97, CDC20, PHGDH, CDC25A, NIPSNAP1, FZD7, PLK1, ..."
9,CSRNP1,7/300,0.0005928,0.0689,5.285,"FOS, TICAM1, RELB, GADD45B, TIPARP, NFKB2, NFKBIE, ..."
10,NFE2L2,7/300,0.0005928,0.0689,5.285,"TICAM1, CDC25A, NPC1, RELB, TIPARP, NFKB2, NFKBIE, ..."


Rank,TF,Overlap,FET p-value,FDR,Odds Ratio,Overlapping Genes
1,MYC,38/3268,3.594e-06,0.000662,2.603,"KDM5B, GOLT1B, TMEM97, GABPB1, EPRS, CDC20, RGS2, NUDCD3, NUSAP1, DAG1, ..."
2,E2F4,33/2762,7.138e-06,0.000662,2.675,"GOLT1B, TMEM97, PYGL, GABPB1, CDC20, NUSAP1, PGM1, ATP6V0B, DNTTIP2, FOS, ..."
3,CREM,51/5083,8.425e-06,0.000662,2.242,"KDM5A, KDM5B, CHIC2, GOLT1B, CCNH, TMEM97, GABPB1, EPRS, PRSS23, TRAM2, ..."
4,EOMES,16/880,9.118e-06,0.000662,4.097,"TMEM97, PYGL, GABPB1, LOXL1, SLC37A4, PGM1, MACF1, CRYZ, KIAA0753, TP53BP2, ..."
5,ZFX,32/2701,1.079e-05,0.000662,2.652,"KDM5B, CHIC2, ARID4B, EPRS, CNDP2, SENP6, NUDCD3, TUBB6, DAG1, PIP4K2B, ..."
6,FOXM1,15/876,3.201e-05,0.00164,3.854,"KDM5A, ARID4B, CDC20, NUSAP1, PIP4K2B, FOS, CDC25A, CDC25B, MTF2, KIF20A, ..."
7,E2F1,36/3465,4.893e-05,0.00188,2.323,"KDM5B, TMEM97, SENP6, NUDCD3, TUBB6, NUSAP1, HADH, SLC25A46, ATP6V0B, TLE1, ..."
8,HIF1A,8/292,8.574e-05,0.00292,6.232,"KDM5B, TRAM2, TLE1, NCOA3, FOS, GADD45B, NFKB2, P4HA2, ..."
9,CLOCK,9/405,0.0001568,0.00481,5.028,"LOXL1, RGS2, FOS, NIPSNAP1, NPC1, EGR1, GADD45B, GADD45A, NET1, ..."
10,CREB1,24/2168,0.0002373,0.00662,2.476,"KDM5A, KDM5B, CCNH, GABPB1, EPRS, NUDCD3, PIP4K2B, SLC37A4, ATP6V0B, TLE1, ..."
